Install required packages

In [2]:
!pip install bitsandbytes transformers peft datasets accelerate SentencePiece gradio

Import necessary libraries and set GPU configuration

In [1]:
import os

# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
import transformers
from transformers import AutoTokenizer, AutoConfig, LlamaForCausalLM, LlamaTokenizer
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training

/home/ayush/anaconda3/envs/myenv/lib/python3.9/site-packages/torch/cuda/__init__.py:107: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0
/home/ayush/anaconda3/envs/myenv/lib/python3.9/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/home/ayush/anaconda3/envs/myenv/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


Set hyperparameters and load pre-trained models

In [ ]:
MICRO_BATCH_SIZE = 2  # Change to 4 for NVIDIA 3090 GPU
BATCH_SIZE = 16
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
EPOCHS = 3  # The original paper uses 3
LEARNING_RATE = 3e-55  # Learning rate from the original paper
CUTOFF_LEN = 400  # The cutoff length, 256 accounts for about 96% of the data

# Load the pre-trained model and tokenizer
model = LlamaForCausalLM.from_pretrained(
    "openlm-research/open_llama_3b",
    load_in_8bit=True,
    device_map="auto",
)
model.tie_weights()

tokenizer = LlamaTokenizer.from_pretrained(
    "openlm-research/open_llama_3b", add_eos_token=True
)

# Prepare the model for int8 training
model = prepare_model_for_int8_training(model)

print(model)


Load and configure dataset

In [ ]:
from datasets import load_dataset

# Set LORA (Loss-aware Relevance Analysis) configuration
LORA_R = 4
LORA_ALPHA = 16
LORA_DROPOUT = 0.05

# Configure LORA for the model
config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)

tokenizer.pad_token_id = 0  # The unk token is set to 0

# Load the dataset
data = load_dataset('PrinceAyush/Mental_Health_conv')


Define a function to generate prompts for the chatbot

In [ ]:
def generate_prompt(data_point):
    if data_point["questionText"]:
        return f"""As a Mental Health Support Chatbot, your primary goal is to provide guidance and support to individuals seeking mental health advice. Your responses should be empathetic, non-judgmental, and focused on promoting emotional well-being.
### Instruction:
{data_point["questionText"]}
### Input:
{data_point["questionTitle"]}
### Response:
{data_point["answerText"]}"""
    else:
        return f"""Below is the instruction.As a Mental Health Support Chatbot, your primary goal is to provide guidance and support to individuals seeking mental health advice. Your responses should be empathetic, non-judgmental, and focused on promoting emotional well-being.
### Instruction:
{data_point["questionTitle"]}
### Response:
{data_point["answerText"]}"""


Process and tokenize the dataset

In [ ]:
data = data.shuffle().map(
    lambda data_point: tokenizer(
        generate_prompt(data_point),
        truncation=True,
        max_length=CUTOFF_LEN,
        padding="max_length",
    )
)


Train the chatbot using the transformers.Trainer class

In [ ]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=MICRO_BATCH_SIZE,
        gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
        warmup_steps=100,
        num_train_epochs=EPOCHS,
        learning_rate=LEARNING_RATE,
        fp16=True,
        logging_steps=1,
        output_dir="Mentalhealth",
        save_total_limit=3,
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False  # Disable model caching

trainer.train(resume_from_checkpoint=True)  # Resume training from the last checkpoint


Save the trained model

In [ ]:
model.save_pretrained("Mentalhealth_chatbot")
